In [2]:
#--------DATA-------#

import matplotlib
matplotlib.use('TkAgg')
import numpy as np
import matplotlib.gridspec as grid
import scipy.misc
import scipy.ndimage
from astropy.convolution import *
import matplotlib.pyplot as plt
import random as rnd
from astropy.io import fits
import sys
import fnmatch
import os
import pylab as py
#import pyfits as pyfit

%matplotlib inline

In [3]:


#---- imput data-----#
np.random.seed(0)
datapoints = 10000
x_data1 = np.random.standard_normal(datapoints) #for i in range (0, datapoints)]
y_data1 = np.random.standard_normal(datapoints) #for i in range (0, datapoints)]

x_data2 = np.random.standard_normal(datapoints)+1
y_data2 = np.random.standard_normal(datapoints)+1

x_data3 = np.random.standard_normal(datapoints)+1
y_data3 = np.random.standard_normal(datapoints)+3

x_data4 = np.random.standard_normal(datapoints)*1.5
y_data4 = np.random.standard_normal(datapoints)*1.5

x_data5 = np.random.standard_normal(datapoints)-1 
y_data5 = np.random.standard_normal(datapoints)-1

x_data6 = np.random.standard_normal(datapoints)-1
y_data6 = np.random.standard_normal(datapoints)-3

x_data = x_data3+x_data2+x_data1+x_data4
y_data = y_data3+y_data2+y_data1+y_data4

H1, xedges, yedges = np.histogram2d(y_data1, x_data1, bins=np.arange(-5.,5.1,0.05))
H2, xedges, yedges = np.histogram2d(y_data2, x_data2, bins=np.arange(-5.,5.1,0.05))
H3, xedges, yedges = np.histogram2d(y_data3, x_data3, bins=np.arange(-5.,5.1,0.05))
H4, xedges, yedges = np.histogram2d(y_data4, x_data4, bins=np.arange(-5.,5.1,0.05))
H5, xedges, yedges = np.histogram2d(y_data5, x_data5, bins=np.arange(-5.,5.1,0.05))
H6, xedges, yedges = np.histogram2d(y_data6, x_data6, bins=np.arange(-5.,5.1,0.05))

# ---- finished mock galaxies
B = np.ones((1000,1000))
lumpy= H1+H2+H3+H4
morelumpy= H1+H2+H3+H4+H5+H6+H1+H2+H3+H4
gaussian = H1
noisygaussian = H1+H4
symmetric = H5+H2

#----------------------##----------------------#
######SDSS-MORPH CODE#######

def aperture_crider(image, radius):
    mask_image = np.copy(image)
    lx, ly = mask_image.shape
    X, Y = np.ogrid[0:lx, 0:ly]
    mask = (X - lx / 2) ** 2 + (Y - ly / 2) ** 2 > radius**2
    mask_image[mask] = 0
    n_pixels = float(lx*ly - mask.sum())
    return float(mask_image.sum()), n_pixels
    
def petrosian_radius(image):
    # http://spiff.rit.edu/classes/phys443/lectures/gal_1/petro/petro.html
    lx, ly = image.shape
    n = min([lx/2, ly/2])

    r = np.arange(n)
    counts = np.arange(n, dtype=np.float)
    pixels = np.arange(n)

    for i in r:
        counts[i], pixels[i] = aperture_crider(image, i)
            
    local_intensity = (counts[1:n-1] - counts[0:n-2])/(pixels[1:n-1] - pixels[0:n-2])
    eta = local_intensity / (counts[1:n-1] / pixels[1:n-1])
    
    # Use ETA cutoff of 0.2 as SDSS does.
    radii = r[np.where(eta<=0.2)]
    petradius=  radii[0]
    return petradius      
#----------------------##----------------------#
#----------------------##----------------------#
def appt(radius,image):
    #line ne possibly taken out when run with stamp!! make function to make stamps of everything !!
    centera,centerb = scipy.ndimage.measurements.center_of_mass(image)
    nx,ny = image.shape
    y,x = np.ogrid[-centera:nx-centera,-centerb:ny-centerb]
    mask = x*x + y*y <= radius*radius
    counts =   sum(image[mask]) 
    return counts

def light_profile(image):
    x,y = image.shape
    n =min([x/2, y/2]) # /2 due to radius calculation
    I=[]
    for i in np.arange(1,n+1.):# start at radius of 1 then work outwards keep 0 for Io??
        I.append(appt(i,image))
    return I

def halflight_radius(image):  
    x,y = image.shape
    I = light_profile(image)
    L = I/I[len(I)-1]
    HL = np.argmax(L>=0.5)
    return HL, L[HL]
       
        #possible problem due to working in counts and not in counts per pixel ?    
def petrosian_radius_own(image): 
    lx =np.arange(image.shape[0]) #lenght of x axis
    ly = np.arange(image.shape[1])#lenght of y axis
    coordinates = [np.meshgrid(lx,ly)]#---->list
    gal_coord = np.where (image>0) 
    #Intensity_total = np.sum(image)
    Intensities = image[coordinates] #galaxy only excludes 0s
    #Intensity_mean =np.mean( image[gal_coord])
    K =light_profile(image)
    r= np.arange(1,len(K),1)
    
    pet_frac=[light_profile(image)[0]]
    for i in r:
        pet_frac.append(np.mean(light_profile(image)[:i+1])/(light_profile[i]-light_profile(image)[i-1]))
       
    pr =np.where(np.array(pet_frac) <=5)# something wrong with cut off !!!
    #petradius=  np.min(pr)
    return pr,pet_frac
    
def smoothness(image):
    original = image
    pr =0.1*petrosian_radius(image)
    smooth = convolve(original,Box2DKernel(pr))
    plt.figure()
    G = grid.GridSpec (2,2)
    axes_1 = py.subplot(G[0, :-1])
    plt.imshow(image, interpolation = 'none')
    plt.title('Original')
    #pl.plot(X,Y,"k:",ms=1,linewidth=2.0)
    axes_2 = py.subplot(G[0, -1])
    plt. imshow(smooth, interpolation = 'none')
    plt.title('Smoothed')
    smoothness_factor = np.sum(np.abs(image-smooth))/np.sum(image)
    #print smoothness_factor
    axes_3 = py.subplot(G[1, :])
    plt.imshow((image-smooth), interpolation ='none')
    plt.show()  
    return smoothness_factor
    
def concentration(image):
    x,y = image.shape
    I = light_profile(image)
    L = I/I[len(I)-1]
    HL = np.argmax(L>=0.5)
    C20 =  np.argmax(L>=0.2)
    C80 =  np.argmax(L>=0.8)
    C=5*np.log10(C80/C20)
    return C
    
def stamp(image):
    center =scipy.ndimage.measurements.center_of_mass(image)
    centery = center[0]
    centerx= center[1]
    #print 'Center',center
# crop image to center on barycenter coordianates
# try to define offset rom real center and then crop from there [miny+Ay:maxy+Ay,minx+Ax:maxx+Ax]

    R_Y=len(yedges)/2
    R_X= len(xedges)/2
    Ax = centerx-R_X
    Ay = centery-R_Y
    #print 'X dimension', centerx, R_X
    #print 'Y dmension', centery, R_Y

    stamp = image[Ay:len(yedges)+Ay, Ax:len(xedges)+Ax]
    return stamp


def Asymetry(image):
    
    R = np.rot90(stamp(image),2)
    Asym =np.sum(np.abs(stamp(image)-R))/np.sum(stamp(image))
     
    plt.figure()
    G = grid.GridSpec (2,2)
    axes_1 = py.subplot(G[0, :-1])
    plt.imshow(image, interpolation = 'none')
    plt.title('Original')
    axes_2 = py.subplot(G[0, -1])
    plt. imshow(R, interpolation = 'none')
    plt.title('Rotated')
    axes_3 = py.subplot(G[1, :])
    plt.imshow(stamp(image)-R, interpolation = 'none')
    plt.show()
    #plt.savefig(str(image)+'.png')
    return Asym
    
    
def Morph(image):
    asymetry = Asymetry(image)
    con = concentration(image)
    halflight =halflight_radius(image)
    profile =light_profile(image)
    smoothed = smoothness(image)
    return asymetry,con, smoothed
    